### Indice
* [ Librarys](#1)
    <br>

* [Dataset](#1)
    <br>
    
* [data_trasnform](#2)

    <br>

* [Study_1:   n_estimators, colsample_bytree](#3)
    <br>
    
    * [Objective Study RandomForest](#1.1)
    * [Best trial]()
    * [Plot_optimization_history](#1.1)
    * [Plot_param_importances]()
    * [plot_parallel_coordinate]()
<br>
<br>
* [Study_2:  eta (learning rate), max_depth, gamma](#3)
    <br>
    
    * [Objective Study RandomForest](#1.1)
    * [Best trial]()
    * [Plot_optimization_history](#1.1)
    * [Plot_param_importances]()
    * [plot_parallel_coordinate]()
<br>
<br>
* [Marge Studys and Save Json](#3)

In [5]:
#Standar Librarys
import pandas as pd
import numpy as np
import scipy
import optuna
from sklearn.metrics import recall_score
from sklearn.metrics import plot_confusion_matrix

# data preparation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

# remove warnings
import warnings
warnings.filterwarnings('ignore')

#utis 
from utils import *

#model Optimizer
from sklearn.naive_bayes import GaussianNB

In [3]:
#================= data===========================#
df = pd.read_csv('../data/bank-additional-full.csv',sep = ';')
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
X,y = data_trasnform(df, smote=False)

    Study_1 , with the Hyperpameters:  n_estimators, colsample_bytree

In [7]:
#==========================OPTUNA -- XGB ===========================#
def objective(trial,data=X,target=y): #objetevie_xgb
    
     #max_depth, n_estiamtoders, max_feature
    
    # hyperparameter 
    param = {
        
        'var_smoothing': trial.suggest_categorical("var_smoothing", np.logspace(0,-9, num=100)), #
    
    }
    
    #==================kfold=====================#
    skf = StratifiedKFold(n_splits=10)
    model_XGB = GaussianNB(**param)
    accuracies = []

    for idx in skf.split(X, y):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = X[train_idx]
        ytrain = y[train_idx]

        xtest = X[test_idx]
        ytest = y[test_idx]

        model_XGB.fit(xtrain,ytrain)
        preds = model_XGB.predict(xtest)
        fold_acc = recall_score(ytest, preds)
        accuracies.append(fold_acc)
        
    return scipy.stats.gmean(accuracies) #media geometrica spipy

In [8]:
study = optuna.create_study(direction='maximize') #maxizer
study.optimize(objective, n_trials=100) #100
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-11-07 17:46:13,329] A new study created in memory with name: no-name-a3a296ab-86ca-4b1d-a8d1-af1271d39536
[I 2022-11-07 17:46:13,855] Trial 0 finished with value: 0.28325200882485835 and parameters: {'var_smoothing': 0.15199110829529336}. Best is trial 0 with value: 0.28325200882485835.
[I 2022-11-07 17:46:14,345] Trial 1 finished with value: 0.3936478436302224 and parameters: {'var_smoothing': 2.310129700083158e-06}. Best is trial 1 with value: 0.3936478436302224.
[I 2022-11-07 17:46:14,870] Trial 2 finished with value: 0.3000975204932422 and parameters: {'var_smoothing': 0.0657933224657568}. Best is trial 1 with value: 0.3936478436302224.
[I 2022-11-07 17:46:15,385] Trial 3 finished with value: 0.33782492992300245 and parameters: {'var_smoothing': 0.001}. Best is trial 1 with value: 0.3936478436302224.
[I 2022-11-07 17:46:15,943] Trial 4 finished with value: 0.49291951681390717 and parameters: {'var_smoothing': 4.328761281083061e-09}. Best is trial 4 with value: 0.49291951681

[I 2022-11-07 17:46:35,887] Trial 42 finished with value: 0.5165084779208805 and parameters: {'var_smoothing': 1.2328467394420635e-09}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:46:36,405] Trial 43 finished with value: 0.4621687354620153 and parameters: {'var_smoothing': 2.848035868435799e-08}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:46:36,924] Trial 44 finished with value: 0.32978789779593115 and parameters: {'var_smoothing': 0.002848035868435802}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:46:37,443] Trial 45 finished with value: 0.2532141092401045 and parameters: {'var_smoothing': 0.43287612810830584}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:46:38,017] Trial 46 finished with value: 0.34890813468518134 and parameters: {'var_smoothing': 0.0001873817422860383}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:46:38,534] Trial 47 finished with value: 0.36142132895109413 and par

[I 2022-11-07 17:46:58,339] Trial 84 finished with value: 0.26496419828813145 and parameters: {'var_smoothing': 0.3511191734215131}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:46:58,879] Trial 85 finished with value: 0.47165922050691494 and parameters: {'var_smoothing': 1.519911082952933e-08}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:46:59,387] Trial 86 finished with value: 0.354756595860313 and parameters: {'var_smoothing': 8.111308307896872e-05}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:46:59,952] Trial 87 finished with value: 0.4351322875906361 and parameters: {'var_smoothing': 1.232846739442066e-07}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:47:00,473] Trial 88 finished with value: 0.41892994553805046 and parameters: {'var_smoothing': 4.3287612810830526e-07}. Best is trial 9 with value: 0.5165084779208805.
[I 2022-11-07 17:47:00,979] Trial 89 finished with value: 0.4496889422902558 and para

Number of finished trials: 100
Best trial: {'var_smoothing': 1.2328467394420635e-09}


In [1]:
import numpy as np
np.logspace(0,-9, num=100)

array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02, 2.31012970e-02, 1.87381742e-02,
       1.51991108e-02, 1.23284674e-02, 1.00000000e-02, 8.11130831e-03,
       6.57933225e-03, 5.33669923e-03, 4.32876128e-03, 3.51119173e-03,
       2.84803587e-03, 2.31012970e-03, 1.87381742e-03, 1.51991108e-03,
       1.23284674e-03, 1.00000000e-03, 8.11130831e-04, 6.57933225e-04,
       5.33669923e-04, 4.32876128e-04, 3.51119173e-04, 2.84803587e-04,
       2.31012970e-04, 1.87381742e-04, 1.51991108e-04, 1.23284674e-04,
       1.00000000e-04, 8.11130831e-05, 6.57933225e-05, 5.33669923e-05,
       4.32876128e-05, 3.51119173e-05, 2.84803587e-05, 2.31012970e-05,
       1.87381742e-05, 1.51991108e-05, 1.23284674e-05, 1.00000000e-05,
      